In [1]:
import pandas as pd
import os

In [2]:
#define path
radiomics_path = "D:\work\ingham-medphys-coding-master\ingham-medphys-coding-master\data\HN_Radiomics.csv"
clinical_data_path = "D:\work\ingham-medphys-coding-master\ingham-medphys-coding-master\data\HN_ClinicalData.csv"

In [3]:
#read data into dataframe
df_clinical_data=pd.read_csv(clinical_data_path)
df_radiomics=pd.read_csv(radiomics_path)

In [4]:
#attact GTV data
df_gtv_radiomics = df_radiomics[df_radiomics["Structure"].str.startswith("GTV")]
df_gtv_radiomics = df_gtv_radiomics.groupby("id")[["VoxelVolume", "SurfaceArea"]].sum()

In [5]:
#merge data into the clininal data
df = df_clinical_data.merge(df_gtv_radiomics, on="id")

In [6]:
df.head()

,id,dataset,index_tumour_location,age_at_diagnosis,biological_sex,performance_status_ecog,overall_hpv_p16_status,clin_t,clin_n,clin_m,...,event_recurrence_metastatic_free_survival,recurrence_metastatic_free_survival_in_days,event_local_recurrence,local_recurrence_in_days,event_locoregional_recurrence,locoregional_recurrence_in_days,event_distant_metastases,distant_metastases_in_days,VoxelVolume,SurfaceArea
0,HN1004,train,oropharynx,56,male,1.0,negative,4,2,0,...,0,3193,0,3193,0,3193,0,3193,122242.927551,20543.958253
1,HN1006,test,oropharynx,63,female,1.0,negative,1,2,0,...,0,1940,0,1940,0,1940,0,1940,17014.503479,7740.937225
2,HN1022,train,oropharynx,56,male,0.0,positive,1,0,0,...,0,1269,0,1269,0,1269,0,1269,35350.799561,10405.482300
3,HN1026,train,larynx,67,male,0.0,NaN,4,0,0,...,1,600,1,600,1,600,0,1315,3713.607788,1623.465837
4,HN1029,train,oropharynx,54,female,0.0,positive,1,2,0,...,0,2101,0,2101,0,2101,0,2101,14416.694641,5353.958555


In [7]:
Train_Data=df[df["dataset"]=="train"]
Test_Data=df[df["dataset"]=="test"]
Train_Data.info()
Test_Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 0 to 136
Data columns (total 30 columns):
id                                                102 non-null object
dataset                                           102 non-null object
index_tumour_location                             102 non-null object
age_at_diagnosis                                  102 non-null int64
biological_sex                                    102 non-null object
performance_status_ecog                           96 non-null float64
overall_hpv_p16_status                            59 non-null object
clin_t                                            102 non-null int64
clin_n                                            102 non-null int64
clin_m                                            102 non-null int64
ajcc_stage                                        102 non-null object
pretreat_hb_in_mmolperlitre                       62 non-null float64
cancer_surgery_performed                          102 non-nu

In [8]:
Train_Data.isnull().sum()
Test_Data.isnull().sum()

id                                                 0
dataset                                            0
index_tumour_location                              0
age_at_diagnosis                                   0
biological_sex                                     0
performance_status_ecog                            4
overall_hpv_p16_status                            13
clin_t                                             0
clin_n                                             0
clin_m                                             0
ajcc_stage                                         0
pretreat_hb_in_mmolperlitre                       14
cancer_surgery_performed                           0
chemotherapy_given                                 0
radiotherapy_total_treat_time                      0
radiotherapy_refgydose_perfraction_highriskgtv     0
radiotherapy_refgydose_total_highriskgtv           0
radiotherapy_number_fractions_highriskgtv          0
event_overall_survival                        

In [9]:
#data analysis
#null training data: performance_status_ecog 6, overall_hpv_p16_status 43, pretreat_hb_in_mmolperlitre 60
# since the null number of the overall_hpv_p16_status and pretreat_hb_in_mmolperlitre is too large for the model training, drop them at first
# only 6 performance_status_ecog is

df_Train_Data = Train_Data.drop(['overall_hpv_p16_status', 'pretreat_hb_in_mmolperlitre'],axis=1)
df_Test_Data = Test_Data.drop(['overall_hpv_p16_status', 'pretreat_hb_in_mmolperlitre'],axis=1)
df_Train_Data_no_status_ecog = df_Train_Data.drop(['performance_status_ecog'],axis=1)
df_Test_Data_no_status_ecog = df_Test_Data.drop(['performance_status_ecog'],axis=1)

In [10]:
#fill null of the performance_status_ecog
#analysis correlation of performance status ecog with other data
df_train=df_Train_Data.dropna()

In [11]:
corr = df_train[["performance_status_ecog", "age_at_diagnosis", "clin_t", "clin_n", "clin_m", "radiotherapy_refgydose_perfraction_highriskgtv", "radiotherapy_refgydose_total_highriskgtv", "overall_survival_in_days"]].corr()

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns 
plt.figure(figsize=(30,8))
sns.heatmap(corr,cmap='coolwarm',annot = True)
plt.show()

<Figure size 3000x800 with 2 Axes>

In [13]:
df_Train_Data.performance_status_ecog.value_counts()

0.0    50
1.0    43
2.0     2
3.0     1
Name: performance_status_ecog, dtype: int64

In [59]:
df_Train_Data['performance_status_ecog'].fillna(df_Train_Data['performance_status_ecog'].value_counts().index[0], inplace=True)
df_Test_Data['performance_status_ecog'].fillna(df_Test_Data['performance_status_ecog'].value_counts().index[0], inplace=True)

In [60]:
#transform category data into numberic data
#index_tumour_location, 
from sklearn.preprocessing import LabelEncoder
# creating instance of labelencoder
labelencoder = LabelEncoder()
cols = ('index_tumour_location', 'biological_sex', 'ajcc_stage', 'cancer_surgery_performed', 'chemotherapy_given')
# process columns, apply LabelEncoder to categorical features into numeric data
for c in cols:
    df_Train_Data[c] = labelencoder.fit_transform(df_Train_Data[c])
    df_Test_Data[c] = labelencoder.fit_transform(df_Test_Data[c])

In [61]:
df_Train_Data.info()
df_Test_Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 0 to 136
Data columns (total 28 columns):
id                                                102 non-null object
dataset                                           102 non-null object
index_tumour_location                             102 non-null int64
age_at_diagnosis                                  102 non-null int64
biological_sex                                    102 non-null int64
performance_status_ecog                           102 non-null float64
clin_t                                            102 non-null int64
clin_n                                            102 non-null int64
clin_m                                            102 non-null int64
ajcc_stage                                        102 non-null int64
cancer_surgery_performed                          102 non-null int64
chemotherapy_given                                102 non-null int64
radiotherapy_total_treat_time                     102 non-null 

In [62]:
x_train = df_Train_Data.drop(['id', 'dataset', 'overall_survival_in_days', 'recurrence_metastatic_free_survival_in_days', 'local_recurrence_in_days'], axis=1)
Y_train = df_Train_Data["overall_survival_in_days"]
x = x_train.values
y = Y_train.values

In [63]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
x_scaled = sc_X.fit_transform(x)
y_scaled = sc_y.fit_transform(y.reshape(-1,1))
X_train,X_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.33, random_state=42)

d:\app\python\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
d:\app\python\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [64]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor

In [65]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
Random=RandomForestRegressor(n_estimators=400)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, mean_absolute_error
from statsmodels.tools import add_constant
import warnings
warnings.filterwarnings('ignore')
RandomReg=Random.fit(X_train, y_train)
train_pred = RandomReg.predict(X_train)
test_pred = RandomReg.predict(X_test)
test_pred = test_pred.reshape(-1,1)
train_pred = train_pred.reshape(-1,1)
print('Train set accuracy score:', mean_absolute_error(y_train, train_pred))
print('Test set accuracy score:', mean_absolute_error(y_test, test_pred))

Train set accuracy score: 0.12114774592854921
Test set accuracy score: 0.29266723074498735


In [70]:
lassoReg=lasso.fit(X_train, y_train)
train_pred = lassoReg.predict(X_train)
test_pred = lassoReg.predict(X_test)
test_pred = test_pred.reshape(-1,1)
train_pred = train_pred.reshape(-1,1)
print('Train set accuracy score:', mean_absolute_error(y_train, train_pred))
print('Test set accuracy score:', mean_absolute_error(y_test, test_pred))

Train set accuracy score: 0.1911201954015121
Test set accuracy score: 0.4368919667496743


In [71]:
ENetReg=ENet.fit(X_train, y_train)
train_pred = ENetReg.predict(X_train)
test_pred = ENetReg.predict(X_test)
test_pred = test_pred.reshape(-1,1)
train_pred = train_pred.reshape(-1,1)
print('Train set accuracy score:', mean_absolute_error(y_train, train_pred))
print('Test set accuracy score:', mean_absolute_error(y_test, test_pred))

Train set accuracy score: 0.19113495064896266
Test set accuracy score: 0.4382517637421543


In [72]:
GBoostReg=GBoost.fit(X_train, y_train)
train_pred = GBoostReg.predict(X_train)
test_pred = GBoostReg.predict(X_test)
test_pred = test_pred.reshape(-1,1)
train_pred = train_pred.reshape(-1,1)
print('Train set accuracy score:', mean_absolute_error(y_train, train_pred))
print('Test set accuracy score:', mean_absolute_error(y_test, test_pred))

Train set accuracy score: 0.04833636448315935
Test set accuracy score: 0.3285062548268302


In [73]:
model_xgbReg=model_xgb.fit(X_train, y_train)
train_pred = model_xgbReg.predict(X_train)
test_pred = model_xgbReg.predict(X_test)
test_pred = test_pred.reshape(-1,1)
train_pred = train_pred.reshape(-1,1)
print('Train set accuracy score:', mean_absolute_error(y_train, train_pred))
print('Test set accuracy score:', mean_absolute_error(y_test, test_pred))

Train set accuracy score: 0.10271549450129475
Test set accuracy score: 0.326714260256958


In [76]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [75]:
test_x = df_Test_Data.drop(['id', 'dataset', 'overall_survival_in_days', 'recurrence_metastatic_free_survival_in_days', 'local_recurrence_in_days'], axis=1).values
text_y = df_Test_Data["overall_survival_in_days"]
test=sc_X.fit_transform(test_x)


In [77]:
y_te_Random = RandomReg.predict(test).reshape(-1,1)
test_prediction =sc_y.inverse_transform(y_te_Random)

In [88]:
np.mean(test_prediction[0,:]-text_y)

578.5914285714281

In [90]:
y_te_lasso = lassoReg.predict(test).reshape(-1,1)
test_prediction =sc_y.inverse_transform(y_te_lasso)
np.mean(test_prediction[0,:]-text_y)

428.85795987566644

In [91]:
y_te_ENet = ENetReg.predict(test).reshape(-1,1)
test_prediction =sc_y.inverse_transform(y_te_ENet)
np.mean(test_prediction[0,:]-text_y)

426.4342771709539

In [92]:
y_te_GBoost = GBoostReg.predict(test).reshape(-1,1)
test_prediction =sc_y.inverse_transform(y_te_GBoost)
np.mean(test_prediction[0,:]-text_y)

579.6081911064508

In [94]:
y_te_xgb = model_xgbReg.predict(test).reshape(-1,1)
test_prediction =sc_y.inverse_transform(y_te_xgb)
np.mean(test_prediction[0,:]-text_y)

231.11801060267857

array([             inf,              inf,              inf,
                    inf,  4.34097461e+133,              inf,
                    inf,              inf, -1.00000000e+000,
                    inf,              inf,              inf,
                    inf,              inf,              inf,
                    inf,              inf,              inf,
                    inf,              inf,              inf,
                    inf,              inf,              inf,
                    inf,              inf,  1.65569341e+088,
                    inf,              inf,              inf,
        6.53454005e+099,              inf,              inf,
                    inf,              inf])